In [9]:
import cvxpy as cp
import numpy as np

# 1. Definir os vértices da incerteza para k e f
k_range = [0.09, 0.4]
f_range = [0.0038, 0.04]

system_vertices = []
for k in k_range:
    for f in f_range:
        # Para cada par (k, f), definir um sistema de vértice
        A_vertices = np.array([[0, 0, 1, 0],
                             [0, 0, 0, 1],
                             [-k, k, -f, f],
                             [k, -k, f, -f]])
        system_vertices.append(A_vertices)

print(f"Analisando {len(system_vertices)} vértices do sistema.")

# Matrizes constantes do sistema
B1 = np.array([[0], [0], [0], [1]])

B2 = np.array([[0], [0], [0], [1]])

C = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 0, 0]])

D = np.array([[0], [0], [1]])

n1, n2, n3 = A_vertices.shape[0], C.shape[0], B1.shape[1]
# Definir as variáveis de otimização
X = cp.Variable((n3, n3), symmetric=True)
W = cp.Variable((n1, n1), symmetric=True)
Z = cp.Variable((1, n1))
rho = cp.Variable()

# 3. Construir a lista de restrições LMI
constraints = [
    cp.trace(X) <= rho,
    W >> 0  # W deve ser positiva definida para ser uma matriz de Lyapunov válida
]

# LMI1 é independente dos vértices, então a adicionamos apenas uma vez
M11 = X
M12 = B1.T
M21 = B1
M22 = W
LMI1 = cp.bmat([[M11, M12], [M21, M22]])
constraints.append(LMI1 >> 0)

# Adicionar LMI2 para cada vértice do sistema
for A_i in system_vertices:
    N11 = A_i @ W + B2 @ Z + W @ A_i.T + Z.T @ B2.T
    N12 = W @ C.T + Z.T @ D.T
    N21 = C @ W + D @ Z  
    N22 = -np.eye(n2)
    LMI2_i = cp.bmat([[N11, N12], [N21, N22]])
    constraints.append(LMI2_i << 0)

# 4. Definir e resolver o problema de otimização
objective = cp.Minimize(rho)
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.SCS)

# 5. Verificar e imprimir a solução
if problem.status in [cp.OPTIMAL, cp.OPTIMAL_INACCURATE]:
    print("\nStatus:", problem.status)
    print(f"Custo H2 garantido (rho): {problem.value:.4f}")

    # O ganho do controlador K = Z * W^-1
    try:
        W_inv = np.linalg.inv(W.value)
        K_feedback = Z.value @ W_inv
        print("\nGanho do controlador K = Z * W^-1:")
        print(K_feedback)
    except np.linalg.LinAlgError:
        print("\nA matriz W não é invertível.")

else:
    print("\nO problema não pôde ser resolvido. Status:", problem.status)
    print("Isso pode significar que não existe um controlador robusto para a faixa de incerteza especificada.")

Analisando 4 vértices do sistema.

Status: optimal_inaccurate
Custo H2 garantido (rho): 3.0904

Ganho do controlador K = Z * W^-1:
[[ 1.80937361 -4.20843452 -9.63095046 -3.08857432]]
